# Sysmon Process Analysis

This notebook is intended to review process creations as revealed by Sysmon logs.

In [1]:
# Import dependencies
import sys
import os
sys.path.append(os.path.abspath("../../utils"))
from malware import *
from sysmon import load_events
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from getpass import getpass
import networkx as nx
from virus_total_apis import PublicApi as VirusTotalPublicApi
import re

Begin by entering the file to examine (relative or absolute path)

In [2]:
file_path = widgets.Text(
    value='examples/HuntingMetasploit.evtx',
    placeholder='/path/to/logfile.evtx',
    description='Logfile',
    disabled=False
)
display(file_path)

Text(value='examples/HuntingMetasploit.evtx', description='Logfile', placeholder='/path/to/logfile.evtx')

In [3]:
VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

Enter VirusTotal API Key (blank if none):  ································································


In [4]:
vt = VirusTotalPublicApi(VT_API_KEY)

In [5]:
# Create DataFrame
events = load_events(file_path.value)
df = pd.DataFrame([vars(e) for e in events])

In [6]:
event_1 = df[df.event_id == "1"]

## Hash Review

Let's start by choosing the images to review

In [7]:
event_1.groupby("image").count().sort_values(by="pid", ascending=False)

,soup,user_id,event_id,time_created,pid,user,src_ip,dest_ip,src_port,dest_port,integrity_levels,hashes,command_line,parent_command_line,parent_image,target_object,details,target_filename,query_name,query_results
image,,,,,,,,,,,,,,,,,,,,
C:\Windows\System32\taskhostw.exe,7,7,7,7,7,7,0,0,0,0,7,7,7,7,7,0,0,0,0,0
C:\Windows\System32\AtBroker.exe,4,4,4,4,4,4,0,0,0,0,4,4,4,4,4,0,0,0,0,0
C:\Windows\System32\LogonUI.exe,4,4,4,4,4,4,0,0,0,0,4,4,4,4,4,0,0,0,0,0
C:\Windows\System32\TSTheme.exe,4,4,4,4,4,4,0,0,0,0,4,4,4,4,4,0,0,0,0,0
C:\Windows\System32\consent.exe,2,2,2,2,2,2,0,0,0,0,2,2,2,2,2,0,0,0,0,0
C:\Windows\System32\rdpclip.exe,2,2,2,2,2,2,0,0,0,0,2,2,2,2,2,0,0,0,0,0
C:\Windows\System32\sethc.exe,2,2,2,2,2,2,0,0,0,0,2,2,2,2,2,0,0,0,0,0
C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2011.6-0\MpCmdRun.exe,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0
C:\Users\THM-Threat\Downloads\shell.exe,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0


In [20]:
review_images = input("Please enter a comma-separated list of images to review: ").split(",")
for_review = event_1[event_1.image.apply(lambda i: i in review_images)].reset_index()
pd.set_option("display.max_colwidth", 100)

Please enter a comma-separated list of images to review:  C:\Users\THM-Threat\Downloads\shell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe


Let's submit these bad boys to VirusTotal!

In [24]:
for idx, h in enumerate(for_review.hashes.values):
    clean_hashes = re.sub(r"[A-Z0-9]+=","", h).split(",") 
    hash_results = []
    for c in clean_hashes:
        res = vt.get_file_report(c)
        hash_results.append(str(malicious_confidence(res)))
        for_review.loc[idx, "malicious_confidence"] = ",".join(hash_results)

In [28]:
for_review[["image","malicious_confidence"]]

,image,malicious_confidence
0,C:\Users\THM-Threat\Downloads\shell.exe,"None,None,None"
1,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,"0.0,None,None"


**Malicious Confidence** is the result of averaging any scan data from Virus Total for each hash provided. `None` means no data was returned, but any number is the percentage of the VT providers that detected the hash as malicious.